In [ ]:
from google.colab import drive
drive.mount('/content/d')

Drive already mounted at /content/d; to attempt to forcibly remount, call drive.mount("/content/d", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

# Reading the  data



In [ ]:
# Reading the meta data
# importing libraries

import gzip
import shutil

# #Path to the meta data zip file. 'sentiment_analysis' is the folder name under 'My Drive'
path1 = '/content/d/MyDrive/CAPSTONE_DATA/meta_Cell_Phones_and_Accessories.json.gz'



# # Path to meta data .json file
path2 = '/content/d/MyDrive/CAPSTONE_DATA/meta_Cell_Phones_and_Accessories.json'

# Unzipping the meta data file
with gzip.open(path1, 'rb') as f_in:
    with open(path2, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# # Reading the unzipped meta data into a Python list. The result will be a list of dictionaries. 
import json

# # Empty list to store the dictonaries
phonemetadata = []

# # Reading the dictionaries in the json file and appending it to the list phonemetadata[]
with open(path2, 'r') as f:
    for line in f:
        phonemetadata.append(json.loads(line))

In [ ]:
# Getting the number of entries in the phonemetadata list
len(phonemetadata)

590071

In [ ]:
#converting the list phonemetadata into a data frame
df_meta=pd.DataFrame(phonemetadata)


In [ ]:
#Reading the .csv file of the phone data into a dataframe

df_phonedata = pd.read_csv('/content/d/MyDrive/CAPSTONE_DATA/Cell_Phones_and_Accessories_5.csv')



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_phonedata.head(5)

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,review_sentiment
0,5.0,True,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN,POSITIVE
1,5.0,True,A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN,POSITIVE
2,3.0,True,AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN,NEGATIVE
3,2.0,True,A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN,POSITIVE
4,4.0,True,A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN,POSITIVE


In [ ]:
# pd.read_csv('Cell_Phones_and_Accessories_5.csv', dtype={'overall': 'float', 'verified': 'Bool', 'reviewerID': 'Bool','asin': 'Bool','verified': 'Bool','verified': 'Bool','verified': 'Bool','verified': 'Bool','verified': 'Bool',  })

In [ ]:
len(df_phonedata)

1128437

In [ ]:
df_meta.head(2)

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,"[Cell Phones & Accessories, Accessories, Scree...",,[],,Trollies Sing a Long Party VHS,[],[],,,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,,,,6303195164
1,"[Cell Phones & Accessories, Accessories, Car A...",,[],,La Pura VHS,[],[],,Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,,,,630524412X


In [ ]:
df_meta.shape

(590071, 18)

In [ ]:
df_meta.drop_duplicates(subset ="asin",keep=False,inplace=True)

In [ ]:
df_meta.shape

(588641, 18)

# Data pre-processing



In [ ]:
# Converting unix review time to date-time format

#Transforming unixReview time to date time format
from datetime import datetime, timedelta
df_phonedata['Date&Time'] = df_phonedata['unixReviewTime'].apply(lambda d: (datetime.fromtimestamp(d) - timedelta(hours=2)).strftime('%Y-%m-%d'))


In [ ]:
df_phonedata.drop(['unixReviewTime'], axis = 1, inplace=True)

In [ ]:
#joining dfs on 'asin' 
df_f01=df_phonedata.merge(df_meta, left_on='asin', right_on='asin', suffixes=('_data', '_meta'))

In [ ]:
df_f01.head(1)

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,vote,image_data,review_sentiment,Date&Time,category,tech1,description,fit,title,also_buy,image_meta,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
0,5.0,True,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,NaN,NaN,POSITIVE,2014-08-03,"[Cell Phones & Accessories, Cases, Holsters & ...",,"[, Elegani Butterfly Case 3D Pattern Back Cove...",,MinisDesign 3d Bling Crystal Bow Transparent C...,[],[https://images-na.ssl-images-amazon.com/image...,,ELEGANI,[Fashionable with unique 3D butterfly design f...,"[>#228,544 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,


In [ ]:
def findstr(x):
  for l in x: 
    if l=='Unlocked Cell Phones' or l=='Carrier Cell Phones': 
      return l 



# EDA

In [ ]:
f1=df_f01['category'].str[2]=='Unlocked Cell Phones' 
f2=df_f01['category'].str[2]=='Carrier Cell Phones' 


In [ ]:
#creating filtered df on above parameters 

dfok=df_f01.loc[f1 | f2]

In [ ]:
len(dfok) 
df_f02=dfok.copy()  #working database for further analysis 

In [ ]:
df_f02.head(2)



,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,vote,image_data,review_sentiment,Date&Time,category,tech1,description,fit,title,also_buy,image_meta,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
566,4.0,False,A1F9Z42CFF9IAY,B00063DKVC,NaN,phototristan,The Sidekick 2 has a better form factor than t...,Better than all the alternatives,19,NaN,POSITIVE,2004-10-06,"[Cell Phones & Accessories, Cell Phones, Carri...",,"[The latest phone features, like an integrated...",,T-MOBILE SIDEKICK II,[],[https://images-na.ssl-images-amazon.com/image...,,,[],"[>#3,090,469 in Cell Phones & Accessories (See...","[B00NSRC4Y2, B00J1SMJ06, B00J7ZKSPQ, B00B4HLLB...",{},Cell Phones & Accessories,"class=""a-bordered a-horizontal-stripes a-spa...",,
567,2.0,False,A5JLAU2ARJ0BO,B00063DKVC,NaN,Old-and-Wise,Sidekick II is meant to be your one-stop-shop ...,"It's the service, stupid",22,NaN,NEGATIVE,2004-10-05,"[Cell Phones & Accessories, Cell Phones, Carri...",,"[The latest phone features, like an integrated...",,T-MOBILE SIDEKICK II,[],[https://images-na.ssl-images-amazon.com/image...,,,[],"[>#3,090,469 in Cell Phones & Accessories (See...","[B00NSRC4Y2, B00J1SMJ06, B00J7ZKSPQ, B00B4HLLB...",{},Cell Phones & Accessories,"class=""a-bordered a-horizontal-stripes a-spa...",,


In [ ]:
df_f02['style'].value_counts()

{'Color:': ' Black'}                                  4944
{'Color:': ' White'}                                  4225
{'Color:': ' Gold'}                                    851
{'Size:': ' 32 GB', 'Color:': ' Black'}                737
{'Color:': ' Blue'}                                    580
                                                      ... 
{'Color:': ' Grey', 'Style:': ' Dash 5.0 +'}             1
{'Color:': ' B606 - White'}                              1
{'Size:': ' Single-SIM', 'Color:': ' Black'}             1
{'Color:': ' White Gold', 'Style:': ' US GSM (U)'}       1
{'Color:': ' White', 'Style:': ' Dash 5.0 +'}            1
Name: style, Length: 609, dtype: int64

In [ ]:
df_f02['style'].isnull().sum()

33265

In [ ]:

df_f02['rank'] = df_f02['rank'].astype(str)

In [ ]:
df_f02['rank']

566        ['>#3,090,469 in Cell Phones & Accessories (Se...
567        ['>#3,090,469 in Cell Phones & Accessories (Se...
568        ['>#3,090,469 in Cell Phones & Accessories (Se...
569        ['>#3,090,469 in Cell Phones & Accessories (Se...
570        ['>#3,090,469 in Cell Phones & Accessories (Se...
                                 ...                        
1124949    ['>#657,463 in Cell Phones & Accessories (See ...
1124950    ['>#657,463 in Cell Phones & Accessories (See ...
1124951    ['>#657,463 in Cell Phones & Accessories (See ...
1124952    ['>#657,463 in Cell Phones & Accessories (See ...
1124953    ['>#657,463 in Cell Phones & Accessories (See ...
Name: rank, Length: 62075, dtype: object

In [ ]:
#cleaning rank comumn 
df_f02['rnk']=df_f02['rank'].str.extract('([0-9]*,[0-9]*,[0-9]*)', expand=False)


In [ ]:
df_f02.head(2)

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,vote,image_data,review_sentiment,Date&Time,category,tech1,description,fit,title,also_buy,image_meta,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,rnk
566,4.0,False,A1F9Z42CFF9IAY,B00063DKVC,NaN,phototristan,The Sidekick 2 has a better form factor than t...,Better than all the alternatives,19,NaN,POSITIVE,2004-10-06,"[Cell Phones & Accessories, Cell Phones, Carri...",,"[The latest phone features, like an integrated...",,T-MOBILE SIDEKICK II,[],[https://images-na.ssl-images-amazon.com/image...,,,[],"['>#3,090,469 in Cell Phones & Accessories (Se...","[B00NSRC4Y2, B00J1SMJ06, B00J7ZKSPQ, B00B4HLLB...",{},Cell Phones & Accessories,"class=""a-bordered a-horizontal-stripes a-spa...",,,"3,090,469"
567,2.0,False,A5JLAU2ARJ0BO,B00063DKVC,NaN,Old-and-Wise,Sidekick II is meant to be your one-stop-shop ...,"It's the service, stupid",22,NaN,NEGATIVE,2004-10-05,"[Cell Phones & Accessories, Cell Phones, Carri...",,"[The latest phone features, like an integrated...",,T-MOBILE SIDEKICK II,[],[https://images-na.ssl-images-amazon.com/image...,,,[],"['>#3,090,469 in Cell Phones & Accessories (Se...","[B00NSRC4Y2, B00J1SMJ06, B00J7ZKSPQ, B00B4HLLB...",{},Cell Phones & Accessories,"class=""a-bordered a-horizontal-stripes a-spa...",,,"3,090,469"


In [ ]:
#extracting number of viewed items 
emp=[]
for i in df_f02['also_view']:
  emp.append(len(i))


df_f02['no_of_viewed_items']=emp



In [ ]:
df_f02.no_of_viewed_items

566        20
567        20
568        20
569        20
570        20
           ..
1124949    13
1124950    13
1124951    13
1124952    13
1124953    13
Name: no_of_viewed_items, Length: 62075, dtype: int64

In [ ]:
df_f02.drop(columns=['rank'],inplace=True)

In [ ]:
df_f02.head(1)

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,vote,image_data,review_sentiment,Date&Time,category,tech1,description,fit,title,also_buy,image_meta,tech2,brand,feature,also_view,details,main_cat,similar_item,date,price,rnk,no_of_viewed_items
566,4.0,False,A1F9Z42CFF9IAY,B00063DKVC,NaN,phototristan,The Sidekick 2 has a better form factor than t...,Better than all the alternatives,19,NaN,POSITIVE,2004-10-06,"[Cell Phones & Accessories, Cell Phones, Carri...",,"[The latest phone features, like an integrated...",,T-MOBILE SIDEKICK II,[],[https://images-na.ssl-images-amazon.com/image...,,,[],"[B00NSRC4Y2, B00J1SMJ06, B00J7ZKSPQ, B00B4HLLB...",{},Cell Phones & Accessories,"class=""a-bordered a-horizontal-stripes a-spa...",,,"3,090,469",20


In [ ]:
df_meta.head(2)

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,"[Cell Phones & Accessories, Accessories, Scree...",,[],,Trollies Sing a Long Party VHS,[],[],,,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,,,,6303195164
1,"[Cell Phones & Accessories, Accessories, Car A...",,[],,La Pura VHS,[],[],,Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,,,,630524412X


In [ ]:
#zipping dictionary of asins and products for later analysis 
asins_and_products=dict(zip(df_meta.asin,df_meta.brand))

In [ ]:
df_f02.isnull().sum()/len(df_f02)

overall               0.000000
verified              0.000000
reviewerID            0.000000
asin                  0.000000
style                 0.535884
reviewerName          0.000145
reviewText            0.000532
summary               0.000290
vote                  0.805107
image_data            0.990270
review_sentiment      0.000000
Date&Time             0.000000
category              0.000000
tech1                 0.000000
description           0.000000
fit                   0.000000
title                 0.000000
also_buy              0.000000
image_meta            0.000000
tech2                 0.000000
brand                 0.000000
feature               0.000000
also_view             0.000000
details               0.000000
main_cat              0.000000
similar_item          0.000000
date                  0.000000
price                 0.000000
rnk                   0.982618
no_of_viewed_items    0.000000
dtype: float64

In [ ]:
df_f02.drop(columns=['style','vote','image_data','rnk'],inplace=True)

In [ ]:
df_f02.shape

(62075, 26)

In [ ]:
#cleaning price col
df_f02['price']=df_f02['price'].str.extract('([0-9]{1,}.[0-9]{1,})', expand=False)


In [ ]:
# filling na values of price with 0 

df_f02['price'] = (
    df_f02['price'].fillna(0)
    .astype(float)
    
    .where(df_f02['price'].notnull())
)

In [ ]:
df_f02.drop(columns=['reviewerName','tech1','description','fit','also_buy','also_view','image_meta','tech2','details','date'],inplace=True)

In [ ]:
#cleaning for further analysis 
df_f02.to_csv('clean4.csv')